In [ ]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [3]:
import minsearch
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = "the course has already started, can I still enroll?"

In [9]:
index.fit(documents)

In [10]:
from openai import OpenAI
import os
#os.environ
api_key = os.getenv('CHATGPT_APIKEY')

In [11]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=api_key,
)

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether or not you can still enroll in a course that has already started depends on several factors, including the policies of the institution or organization offering the course, the instructor’s discretion, and the structure of the course itself.\n\nHere are some steps you can take to find out if late enrollment is possible:\n\n1. **Check Course Policies:** Review the course or institution's policies regarding late enrollment, which are often found on their website or in the course catalog.\n\n2. **Contact the Instructor:** Reach out directly to the instructor or course coordinator to explain your situation and ask if it's still possible to join the class. Be sure to express your interest and commitment to catching up if allowed to enroll late.\n\n3. **Consult the Registration Office:** If you are trying to enroll in a course at an educational institution, contact the registrar or student services office for guidance on the formal process and any deadlines.\n\n4. **Online Course Pla

In [13]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [16]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
rag(query)

'To run Kafka, the steps depend on whether you\'re running it for a Java application or a Python application. Here’s how you can do both:\n\n**For Java:**\n1. Navigate to the project directory.\n2. Execute the following command in the terminal to run the producer, consumer, or kstreams:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Replace `<jar_name>` with the actual name of the jar file.\n\n**For Python:**\n1. Create a virtual environment (this only needs to be done once):\n   ```sh\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n2. To activate the virtual environment (run this each time you need to use the environment):\n   ```sh\n   source env/bin/activate\n   ```\n3. To deactivate the virtual environment after you\'re done:\n   ```sh\n   deactivate\n   ```\n\nFor MacOS and Linux, the virtual environment activation script is `env/bin/activate`, while for Wi

In [18]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You will be eligible to submit the homework assignments, but remember that there will be deadlines for submitting the final projects, so it's important not to leave everything to the last minute."

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [1]:
from elasticsearch import Elasticsearch

In [25]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '09ff6d0aeab9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'm4TLmq7NQsasQc5EeKIWqw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/N0Mhap-LTwGnvVnf7uzAsQ] already exists')

In [27]:
from tqdm.auto import tqdm

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:24<00:00, 38.53it/s]


In [30]:
query = "I just discovered the course. can I still join?"

In [35]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [36]:
response = es_client.search(index=index_name, body=search_query)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7a9d4d491c30>: Failed to establish a new connection: [Errno 111] Connection refused))

In [37]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

TypeError: 'ChatCompletion' object is not subscriptable